<a href="https://www.kaggle.com/code/aisuko/preparing-gnndelete-parmeters?scriptVersionId=197394374" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Overview

In notebook [Loading data for gnndelete](https://www.kaggle.com/code/aisuko/loading-data-for-gnndelete), we failed to manipulate data. However, we deal with them by using RMIT race hub aws service. Currently, we release these data on:

* https://www.kaggle.com/models/skywardai/ogbl-collab
* https://www.kaggle.com/models/skywardai/ogbl-citation2

I released them as models due to easier for us to do testing on Kaggle. They aren't models. Please noticed.

An also, we will twisted the code from GNNdelete project https://github.com/mims-harvard/GNNDelete.

# What we do here?

In this notebook, we will learn and save all the parameters to json file. And we can load all these parameters in the other notebooks.

In [1]:
import argparse

num_edge_type_mapping = {
    'FB15k-237': 237,
    'WordNet18': 18,
    'WordNet18RR': 11,
    'ogbl-biokg': 51
}

In [2]:
num_edge_type_mapping

{'FB15k-237': 237, 'WordNet18': 18, 'WordNet18RR': 11, 'ogbl-biokg': 51}

In [4]:
import json

# Specify the path of the JSON file
file_path = 'data.json'

# Write the dictionary to the JSON file
with open(file_path, 'w') as json_file:
    json.dump(num_edge_type_mapping, json_file, indent=4)  # 'indent=4' makes the JSON file more readable

# Acknowledgement

* https://github.com/SkywardAI/GNNDelete/blob/main/framework/training_args.py